# Download subtitles from Open Subtitle
From the list of movies (represented by the IMDB id), this notebook allows to download .srt subtitle files

In [ ]:
import urllib.parse
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

from datetime import datetime

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from pythonopensubtitles.opensubtitles import OpenSubtitles
from pythonopensubtitles.utils import File

In [ ]:
FOLDER_IMG = "/home/tanguy/data/movizz/img"
FOLDER_CSV = "/home/tanguy/data/movizz/csv"
FOLDER_SRT = '/home/tanguy/data/movizz/srt'

In [ ]:
ost = OpenSubtitles() 

# Your login/password from open subtitle
login = 'YOUR***LOGIN'
password = 'YOUR***PASSWORD'

ost.login(login, password)

### From a imdb id, get a subtitle from opensubtitle

In [ ]:
df_movies = pd.read_csv(os.path.join(FOLDER_CSV, 'metadata_movies.csv'), sep=';', index_col=0)

In [ ]:
COUNT_DL = 0
count_echec = 0
for ttmovie_id in df_movies.index:
    list_folder_srt = [filename for filename in [files for root, dirs, files in os.walk(FOLDER_SRT)]][0]
    
    movie_id = ttmovie_id[2:]
    print(movie_id, end=' ')
    
    if f'{ttmovie_id}.srt' in list_folder_srt:
        print('[OK - already dl]')
        continue

    # Get ID of subtitle file from IMDB ID
    data = ost.search_subtitles([{'sublanguageid': 'fre', 'imdbid':movie_id}])
    data = [d for d in data if d['SubFormat'] == 'srt']

    if len(data) == 0:
        print('[----------- ECHEC (data 0) -----------]')
        continue
    
    id_subtitle = data[0]['IDSubtitleFile']
    
    # Download subtitle file from subtitle ID
    for i in range(2):
        while True:
            try:
                res = ost.download_subtitles([id_subtitle], output_directory=FOLDER_SRT, extension='srt')
            except ProtocolError:
                ost.login(login, password)
            break
        
    COUNT_DL += 1
    # Cancels after 5 attempts
    if res == None:
        if count_echec == 5:
            print('[----------- ECHEC (download 5 times) -----------]')
            break
        count_echec += 1
        print('[----------- ECHEC (download) -----------]')
        continue
#    TODO : Check if 'SubFormat' == 'srt'

    file_name_s = f'{id_subtitle}.srt'
    file_name_t = f'{ttmovie_id}.srt'
    os.rename(os.path.join(FOLDER_SRT,file_name_s), os.path.join(FOLDER_SRT,file_name_t))
    print('[OK]')
